In [ ]:
import pickle
import numpy
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
from numpy.linalg import lstsq
import os
import re

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim

In [ ]:
eig_values = pickle.load(open('../eigen_values_all.pkl', 'rb'))
eig_vector_receiver, eig_vector_source = pickle.load(open('../eigen_vectors_all.pkl', 'rb'))
modes = pickle.load(open("../evaluated_modes_all.pkl", 'rb'))

In [ ]:
print(eig_vector_receiver.shape)
print(eig_vector_source.shape)
print(modes.shape)

In [ ]:
eig_values

In [ ]:
S = np.sum(np.abs(eig_values)**2)

In [ ]:
print(S)

In [ ]:
plt.plot(np.cumsum(np.abs(eig_values[0:50])**2)/S)

In [ ]:
modes = np.asarray([np.reshape(i, (166,166)) for i in modes])

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(15,15))

for i,mode in enumerate(modes[0:9]):
    im1 = ax[i//3][i%3].imshow(mode.real, cmap='jet')
    ax[i//3][i%3].set_title(f"Mode {i}", fontsize=18)
    ax[i//3][i%3].grid(False)
    ax[i//3][i%3].axis('off')
    divider = make_axes_locatable(ax[i//3][i%3])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=14)
    if i == 8:
        break
plt.tight_layout()
fig.savefig("plotted_modes.png", dpi=300)

In [ ]:
meep_data = h5py.File('../../../data/meep-dataset-v2/0001/dft_00001.h5', 'r')

In [ ]:
fields = meep_data['ey_2.r'][:] + 1j*meep_data['ey_2.i'][:]

In [ ]:
field = fields[:,:,-1]
field = field.flatten()

In [ ]:
reshaped_field = np.reshape(field, (166,166))
norm_factor = np.max(np.abs(reshaped_field))
reshaped_field = reshaped_field

min_real = np.min(reshaped_field.real)
max_real = np.max(reshaped_field.real)

min_imag = np.min(reshaped_field.imag)
max_imag = np.max(reshaped_field.imag)

min_abs = np.min(np.abs(reshaped_field))
max_abs = np.max(np.abs(reshaped_field))

min_phase = np.min(np.angle(reshaped_field))
max_phase = np.max(np.angle(reshaped_field))

In [ ]:
fig,ax = plt.subplots(2,2, figsize=(10,10))

im1 = ax[0][0].imshow(reshaped_field.real, cmap='jet', vmin=min_real, vmax=max_real)
divider = make_axes_locatable(ax[0][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[0][1].imshow(reshaped_field.imag, cmap='jet', vmin=min_imag, vmax=max_imag)
divider = make_axes_locatable(ax[0][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[1][0].imshow(np.abs(reshaped_field), cmap='jet', vmin=min_abs, vmax=max_abs)
divider = make_axes_locatable(ax[1][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[1][1].imshow(np.angle(reshaped_field), cmap='hsv', vmin=-np.pi, vmax=np.pi)
divider = make_axes_locatable(ax[1][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

ax[0][0].set_title("Real", fontsize=18)
ax[0][1].set_title("Imaginary", fontsize=18)
ax[1][0].set_title("Magnitude", fontsize=18)
ax[1][1].set_title("Phase", fontsize=18)


for a in ax.flatten():
    a.axis('off')
plt.tight_layout()
fig.savefig("meep_00001.png", dpi=300)

In [ ]:
c = np.conj(eig_vector_receiver.T) @ field

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,3))
ax.plot(np.abs(c[0:20]), color='darkgreen', marker='o')
ax.grid()
ax.set_xlabel("Mode number", fontsize=18)
ax.set_ylabel(r"$|c_m|$", fontsize=18)
plt.tight_layout()
fig.savefig("mode_projection_values.png", dpi=300)

In [ ]:
E_reconstructed = (eig_vector_receiver @ c)
E_reconstructed = E_reconstructed

In [ ]:
E_reshaped = E_reconstructed.reshape(166,166)

In [ ]:
fig,ax = plt.subplots(2,2, figsize=(10,10))

im1 = ax[0][0].imshow(E_reshaped.real, cmap='jet', vmin=min_real, vmax=max_real)
divider = make_axes_locatable(ax[0][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[0][1].imshow(E_reshaped.imag, cmap='jet', vmin=min_imag, vmax=max_imag)
divider = make_axes_locatable(ax[0][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[1][0].imshow(np.abs(E_reshaped), cmap='jet', vmin=min_abs, vmax=max_abs)
divider = make_axes_locatable(ax[1][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[1][1].imshow(np.angle(E_reshaped), cmap='hsv', vmin=-np.pi, vmax=np.pi)
divider = make_axes_locatable(ax[1][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

ax[0][0].set_title("Real", fontsize=18)
ax[0][1].set_title("Imaginary", fontsize=18)
ax[1][0].set_title("Magnitude", fontsize=18)
ax[1][1].set_title("Phase", fontsize=18)


for a in ax.flatten():
    a.axis('off')
plt.tight_layout()
fig.savefig("reconstructed_00001_all.png", dpi=300)

In [ ]:
difference = reshaped_field - E_reshaped
diff_mag = np.abs(reshaped_field) - np.abs(E_reshaped)
diff_phase = np.angle(reshaped_field) - np.angle(E_reshaped)
diff_phase = (diff_phase + np.pi) % (2 * np.pi) - np.pi
max_real_diff = np.max(np.abs(difference.real))
max_imag_diff = np.max(np.abs(difference.imag))
max_abs_diff = np.max(np.abs(diff_mag))

In [ ]:
fig,ax = plt.subplots(2,2, figsize=(10,10))

im1 = ax[0][0].imshow(difference.real, cmap='jet', vmin=-max_real_diff, vmax=max_real_diff)
divider = make_axes_locatable(ax[0][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[0][1].imshow(difference.imag, cmap='jet', vmin=-max_imag_diff, vmax=max_imag_diff)
divider = make_axes_locatable(ax[0][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[1][0].imshow(diff_mag, cmap='jet', vmin=-max_abs_diff, vmax=max_abs_diff)
divider = make_axes_locatable(ax[1][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

im1 = ax[1][1].imshow(diff_phase, cmap='hsv', vmin=-np.pi, vmax=np.pi)
divider = make_axes_locatable(ax[1][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

ax[0][0].set_title("Real", fontsize=18)
ax[0][1].set_title("Imaginary", fontsize=18)
ax[1][0].set_title("Magnitude", fontsize=18)
ax[1][1].set_title("Phase", fontsize=18)


for a in ax.flatten():
    a.axis('off')
plt.tight_layout()
fig.savefig("difference_00001_all.png", dpi=300)

## Okay, let's sweep the number of modes used and for more of the dataset

In [ ]:
path_meep_data = '../../../data/meep-dataset-v2/'
folders = os.listdir(path_meep_data)
folders.sort()

In [ ]:
fields = []
# Only include directories that end with a number
numeric_dirs = [
    os.path.join(path_meep_data, d) 
    for d in folders 
    if os.path.isdir(os.path.join(path_meep_data, d)) and re.match(r'^\d+$', d)
]
print(numeric_dirs)
# Iterate through numeric directories and read H5 files
for d in numeric_dirs:
    for fname in os.listdir(d):
        if re.match(r'^dft_\d+\.h5$', fname):
            h5_path = os.path.join(d, fname)
            with h5py.File(h5_path, 'r') as f:
                # Do something with the file
                fields.append(f['ey_2.r'][:] + 1j*f['ey_2.i'][:])

In [ ]:
fields = [i[:,:,-1].flatten() for i in fields]

In [ ]:
projections = []
for num_modes in np.arange(1, 101, 2):
    temp = []
    for field in fields:
        temp.append(np.conj(eig_vector_receiver[:, 0:num_modes].T) @ field)
    projections.append(np.asarray(temp))

In [ ]:
projections[1].shape

In [ ]:
reconstructions = []
for p in projections:
    num_modes = p.shape[-1]
    temp = []
    for i,f in enumerate(fields):
        # Get the projection for that field
        c = p[i]
        # Get the correct number of eigenvectors
        eig = eig_vector_receiver[:,0:num_modes]
        # Reconstruct
        temp.append(eig @ c)
    reconstructions.append(np.asarray(temp))

In [ ]:
# Reshape the reconstructions
reshaped_recon = []
for i, recon in enumerate(reconstructions):
    temp = []
    for j,r in enumerate(recon):
        temp.append(r.reshape(166,166))
    reshaped_recon.append(np.asarray(temp))

In [ ]:
fig,ax = plt.subplots(4,2,figsize=(10,20))

for j,i in enumerate(np.arange(0, 20, 5)):
    im1 = ax[j][0].imshow(reshaped_recon[i][1].real, cmap='jet', vmin=min_real, vmax=max_real)
    divider = make_axes_locatable(ax[j][0])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=14)
    ax[j][0].set_title("Real", fontsize=18)
    
    im1 = ax[j][1].imshow(reshaped_recon[i][1].imag, cmap='jet', vmin=min_imag, vmax=max_imag)
    divider = make_axes_locatable(ax[j][1])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=14)
    ax[j][1].set_title("Imaginary", fontsize=18)

    print(i)
for a in ax.flatten():
    a.axis('off')

plt.tight_layout()
fig.savefig("reconstructions_increasing_modes.png", dpi=300)

## Okay, now lets calculate MSE, PSNR and SSIM for the reconstructions as a function of number of modes used for recon

In [ ]:
# [num_modes, num_reconstructions, mse, psnr, ssim]

final_mse = []
final_psnr = []
final_ssim = []

for m in range(len(reconstructions)):
    mse_vals = []
    psnr_vals = []
    ssim_vals = []
    for i, recon in enumerate(reconstructions[m]):
        truth = fields[i]

        mse_vals.append([mse(truth.real, recon.real), mse(truth.imag, recon.imag)])
        psnr_vals.append([psnr(truth.real, recon.real), psnr(truth.imag, recon.imag)])
        #ssim_vals.append([ssim(truth.real, recon.real), ssim(truth.imag, recon.imag)])
        
    final_mse.append(np.asarray(mse_vals))
    final_psnr.append(np.asarray(psnr_vals))
    #final_ssim.append(np.asarray(ssim_vals))
final_mse = np.asarray(final_mse)
final_psnr = np.asarray(final_psnr)
#final_ssim = np.asarray(final_ssim)

In [ ]:
print(final_mse.shape)
print(final_psnr.shape)

In [ ]:
# Lets get the average mse for the real and imag
average_mse = final_mse.mean(axis=1)
average_psnr = final_psnr.mean(axis=1)

mse_std = final_mse.std(axis=1)
psnr_std = final_psnr.std(axis=1)

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(15,5))
ax[0].plot(average_mse[:,0], label='MSE real')
ax[0].plot(average_mse[:,1], label='MSE imag')

ax[1].plot(average_psnr[:, 0], label='PSNR real')
ax[1].plot(average_psnr[:, 1], label='PSNR image')

ax[0].set_ylabel("MSE")
ax[1].set_ylabel("PSNR")

ax[0].set_xlabel("Number of modes")
ax[1].set_xlabel("Number of modes")

ax[0].legend()
ax[1].legend()

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(15,5))
ax[0].plot(average_mse[:,0], label='MSE real')
ax[0].plot(average_mse[:,1], label='MSE imag')

ax[1].plot(average_psnr[:, 0], label='PSNR real')
ax[1].plot(average_psnr[:, 1], label='PSNR image')

ax[0].set_ylabel("MSE")
ax[1].set_ylabel("PSNR")

ax[0].set_xlabel("Number of modes")
ax[1].set_xlabel("Number of modes")

ax[0].set_yscale('log')
ax[1].set_yscale('log')
ax[0].legend()
ax[1].legend()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
cumulative_sum = np.cumsum(np.abs(eig_values)**2)
x = np.arange(average_mse.shape[0])  # e.g., 0 to 49

# MSE real
ax[0].plot(x, average_mse[:, 0], label='MSE real', color='darkgreen', linewidth=2)
ax[0].fill_between(x,
                   average_mse[:, 0] - mse_std[:, 0],
                   average_mse[:, 0] + mse_std[:, 0],
                   alpha=0.3, color='darkgreen')



# MSE imag
ax[0].plot(x, average_mse[:, 1], label='MSE imag', color='darkblue', linewidth=2)
ax[0].fill_between(x,
                   average_mse[:, 1] - mse_std[:, 1],
                   average_mse[:, 1] + mse_std[:, 1],
                   alpha=0.3, color='darkblue')

ax2 = ax[0].twinx()
ax2.plot(x, cumulative_sum[0:len(average_mse)], label=r"$\sum_M |S_m|^2$", color='purple', linewidth=2)
ax[0].plot([],[], color='purple', label=r"$\sum_M |S_m|^2$", linewidth=2)

# PSNR real
ax[1].plot(x, average_psnr[:, 0], label='PSNR real', color='darkgreen', linewidth=2)
ax[1].fill_between(x,
                   average_psnr[:, 0] - psnr_std[:, 0],
                   average_psnr[:, 0] + psnr_std[:, 0],
                   alpha=0.3, color='darkgreen')

# PSNR imag
ax[1].plot(x, average_psnr[:, 1], label='PSNR imag', color='darkblue', linewidth=2)
ax[1].fill_between(x,
                   average_psnr[:, 1] - psnr_std[:, 1],
                   average_psnr[:, 1] + psnr_std[:, 1],
                   alpha=0.3, color='darkblue')

ax3 = ax[1].twinx()
ax3.plot(x, cumulative_sum[0:len(average_mse)], label=r"$\sum_M |S_m|^2$", color='purple', linewidth=2)
ax[1].plot([],[], color='purple', label=r"$\sum_M |S_m|^2$", linewidth=2)


threshold_idx = np.argmax(cumulative_sum[0:50]/S >= 0.9)
ax[0].axvline(x[threshold_idx], color='red', linestyle=':', label='90% Energy', linewidth=2)
ax[1].axvline(x[threshold_idx], color='red', linestyle=':', label='90% Energy', linewidth=2)


# Labels and legend
ax[0].set_ylabel("MSE", fontsize=14)
ax[1].set_ylabel("PSNR", fontsize=14)
ax[0].set_xlabel("Number of modes", fontsize=14)
ax[1].set_xlabel("Number of modes", fontsize=14)
ax[0].legend(loc='center right', fontsize=14)
ax[1].legend(loc='center right', fontsize=14)
ax3.set_ylabel("Cumulative mode energy", fontsize=14)
ax2.set_ylabel("Cumulative mode energy", fontsize=14)
plt.tight_layout()
fig.savefig("linear_mse_psnr_byMode.png", dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
cumulative_sum = np.cumsum(np.abs(eig_values)**2)
x = np.arange(average_mse.shape[0])  # e.g., 0 to 49

# MSE real
ax[0].plot(x, average_mse[:, 0], label='MSE real', color='darkgreen', linewidth=2)
ax[0].fill_between(x,
                   average_mse[:, 0] - mse_std[:, 0],
                   average_mse[:, 0] + mse_std[:, 0],
                   alpha=0.3, color='darkgreen')



# MSE imag
ax[0].plot(x, average_mse[:, 1], label='MSE imag', color='darkblue')
ax[0].fill_between(x,
                   average_mse[:, 1] - mse_std[:, 1],
                   average_mse[:, 1] + mse_std[:, 1],
                   alpha=0.3, color='darkblue')

ax2 = ax[0].twinx()
ax2.plot(x, cumulative_sum[0:len(average_mse)], label=r"$\sum_M |S_m|^2$", color='purple', linewidth=2)
ax[0].plot([],[], color='purple', label=r"$\sum_M |S_m|^2$", linewidth=2)

# PSNR real
ax[1].plot(x, average_psnr[:, 0], label='PSNR real', color='darkgreen', linewidth=2)
ax[1].fill_between(x,
                   average_psnr[:, 0] - psnr_std[:, 0],
                   average_psnr[:, 0] + psnr_std[:, 0],
                   alpha=0.3, color='darkgreen')

# PSNR imag
ax[1].plot(x, average_psnr[:, 1], label='PSNR imag', color='darkblue', linewidth=2)
ax[1].fill_between(x,
                   average_psnr[:, 1] - psnr_std[:, 1],
                   average_psnr[:, 1] + psnr_std[:, 1],
                   alpha=0.3, color='darkblue')

ax3 = ax[1].twinx()
ax3.plot(x, cumulative_sum[0:len(average_mse)], label=r"$\sum_M |S_m|^2$", color='purple', linewidth=2)

ax[1].plot([],[], color='purple', label=r"$\sum_M |S_m|^2$", linewidth=2)

threshold_idx = np.argmax(cumulative_sum[0:50]/S >= 0.9)
ax[0].axvline(x[threshold_idx], color='red', linestyle=':', label='90% Energy', linewidth=2)
ax[1].axvline(x[threshold_idx], color='red', linestyle=':', label='90% Energy', linewidth=2)


ax[0].set_yscale('log')
ax[1].set_yscale('log')

# Labels and legend
ax[0].set_ylabel("MSE", fontsize=14)
ax[1].set_ylabel("PSNR", fontsize=14)
ax[0].set_xlabel("Number of modes", fontsize=14)
ax[1].set_xlabel("Number of modes", fontsize=14)
ax[0].legend(loc='center right', fontsize=14)
ax[1].legend(loc='center right', fontsize=14)
ax3.set_ylabel("Cumulative mode energy", fontsize=14)
ax2.set_ylabel("Cumulative mode energy", fontsize=14)

plt.tight_layout()
fig.savefig("log_mse_pnsr_byMode.png", dpi=300)

In [ ]:
cumulative_sum = np.cumsum(np.abs(eig_values)**2)
cumulative_sum /= S
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

x = cumulative_sum[0:len(average_mse)]  # e.g., 0 to 49

# MSE real
ax[0].plot(x, average_mse[:, 0], label='MSE real')
ax[0].fill_between(x,
                   average_mse[:, 0] - mse_std[:, 0],
                   average_mse[:, 0] + mse_std[:, 0],
                   alpha=0.3)

# MSE imag
ax[0].plot(x, average_mse[:, 1], label='MSE imag')
ax[0].fill_between(x,
                   average_mse[:, 1] - mse_std[:, 1],
                   average_mse[:, 1] + mse_std[:, 1],
                   alpha=0.3)

# PSNR real
ax[1].plot(x, average_psnr[:, 0], label='PSNR real')
ax[1].fill_between(x,
                   average_psnr[:, 0] - psnr_std[:, 0],
                   average_psnr[:, 0] + psnr_std[:, 0],
                   alpha=0.3)

# PSNR imag
ax[1].plot(x, average_psnr[:, 1], label='PSNR imag')
ax[1].fill_between(x,
                   average_psnr[:, 1] - psnr_std[:, 1],
                   average_psnr[:, 1] + psnr_std[:, 1],
                   alpha=0.3)

threshold_idx = np.argmax(x >= 0.9)
ax[0].axvline(x[threshold_idx], color='red', linestyle=':', label='90% Energy')
ax[1].axvline(x[threshold_idx], color='red', linestyle=':', label='90% Energy')


# Labels and legend
ax[0].set_ylabel("MSE")
ax[1].set_ylabel("PSNR")
ax[0].set_xlabel(r"$\sum_M |S_m|^2$")
ax[1].set_xlabel(r"$\sum_M |S_m|^2$")

ax[0].legend()
ax[1].legend()

In [ ]:
plt.plot(cumulative_sum[0:50])

In [ ]:
meep_data = h5py.File('../../../data/meep-dataset-v2/0000/dft_00000.h5', 'r')
fields = meep_data['ey_2.r'][:] + 1j*meep_data['ey_2.i'][:]

In [ ]:
plt.imshow(np.angle(fields[:,:,0].real + 1j*fields[:,:,0].imag))